검증 세트

In [9]:
import pandas as pd
import numpy as np
wine=pd.read_csv('https://bit.ly/wine_csv_data')
#와인데이터 불러오기

In [10]:
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy() #데이터 넘파이 배열로 바꾸기

In [11]:
from sklearn.model_selection import train_test_split
train_input,test_iput,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)
#훈련, 테스트 데이터 나누기

In [12]:
sub_input,val_input,sub_target,val_target=train_test_split(train_input,train_target,test_size=0.2,random_state=42)
#검증 세트 생성


In [13]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target)," ",dt.score(val_input,val_target))
#모델 학습이후 훈련세트 점수와 검증세트 점수 출력


0.9971133028626413   0.864423076923077


교차검증

In [15]:
from sklearn.model_selection import cross_validate
scores=cross_validate(dt,train_input,train_target)
print(scores)
print(np.mean(scores['test_score']))
#교차검증,검증 평균값 출력

{'fit_time': array([0.00800109, 0.0080018 , 0.00700164, 0.0080018 , 0.00700116]), 'score_time': array([0.        , 0.00100017, 0.00100017, 0.00200057, 0.00100088]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


In [16]:
from sklearn.model_selection import StratifiedKFold
splitter=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)#분할기 지정
scores=cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))#분할기를 이용해 교차 검증 수행

0.8574181117533719


하이퍼 파라미터 튜닝

In [17]:
from sklearn.model_selection import GridSearchCV
params={'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)
#그리드 서치를 불러오고 파라미터를 설정함 그리고 그리드 서치 모델에다
#자신이 쓸 머신러닝 모델, 그에대한 파라미터와 작업을 수행할 프로세서 입력
#모델 학습

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [18]:
dt=gs.best_estimator_#가장 점수가 좋았던 매개변수 모델 설정
print(dt.score(test_iput,test_target))

0.8653846153846154


In [19]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [20]:
best_index=np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


랜덤 서치

In [21]:
from scipy.stats import uniform,randint
#사이파이에서 정수를 뽑는 randint와
#실수만 뽑는 uniform을 불러옴

In [22]:
rgen=randint(0,10)#리젠변수는 0부터 9까지지정
rgen.rvs(10)#10개만 뽑음
ugen=uniform(0,1)#0<=ugen<1
ugen.rvs(10)#10개만 무작위로 뽑음

array([0.72090276, 0.87453136, 0.44947564, 0.33758461, 0.67751556,
       0.14369375, 0.48584666, 0.88918009, 0.78354395, 0.22461849])

In [23]:
params={'min_impurity_decrease':uniform(0.0001,0.001),
        'max_depth':randint(20,50),
        'min_samples_split':randint(2,25),
        'min_samples_leaf':randint(1,25)}#파라미터 지정

In [28]:
from sklearn.model_selection import RandomizedSearchCV
gs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42)
#spliter='random'으로 하면 엑스트라 트리를 사용한다.
gs.fit(train_input,train_target)
print(gs.best_estimator_)
#랜덤서치후 가장 잘된 모델의 파라 미터 출력

DecisionTreeClassifier(max_depth=39,
                       min_impurity_decrease=0.00034102546602601173,
                       min_samples_leaf=7, min_samples_split=13,
                       random_state=42)


In [29]:
dt=gs.best_estimator_
print(dt.score(test_iput,test_target))

0.86
